In [1]:
!pip install biogeme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cythonbiogeme: filename=cythonbiogeme-1.0.1-cp310-cp310-linux_x86_64.whl size=5504222 sha256=0948bbb7cb69f5160e624f6a97593fde202d555e9cf16bd3a907de8d83cc51a4
  Stored in directory: /root/.cache/pip/wheels/43/d4/07/0d02caa7b6e1280bc54246647cc7d00af17c86875448458947
Successfully built cythonbiogeme


In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db  
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, log, Derive
import biogeme.results as res 
from biogeme import models, tools
import matplotlib.pyplot as plt
import seaborn as sns

## UPLOAD DATA

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [4]:
# Read the data
df = pd.read_csv('dataset.csv')

In [6]:
df['choice'] = df['travel_mode'].astype('category').cat.codes

for elem in set(df["purpose"]):
    df[elem] = np.where(df["purpose"]==elem,1,0)

for elem in set(df["fueltype"]):
    df[elem] = np.where(df["fueltype"]==elem,1,0)

for elem in set(df["faretype"]):
    df[elem] = np.where(df["faretype"]==elem,1,0)

df.drop('faretype',inplace=True,axis=1)
df.drop('purpose',inplace=True,axis=1)
df.drop('fueltype',inplace=True,axis=1)
df.drop('travel_mode',inplace=True,axis=1)

In [7]:
df['choice'].value_counts()

1    35808
2    28605
3    14268
0     2405
Name: choice, dtype: int64

In [8]:
df["no_inter"] = np.where(df["pt_n_interchanges"]== 0,1,0)
df["one_inter"] = np.where(df["pt_n_interchanges"]== 1,1,0)
df["more_than_one_inter"] = np.where(df["pt_n_interchanges"]> 1,1,0)

In [9]:
df.drop('age', inplace = True, axis = 1)

In [10]:
database = db.Database('dataset', df)
globals().update(database.variables)

In [11]:
df

,trip_id,household_id,person_n,trip_n,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,...,Diesel_Car,Diesel_LGV,16+,dis,child,free,full,no_inter,one_inter,more_than_one_inter
0,0,0,0,0,1.0,1,2012,4,1,7,...,0,0,0,0,0,0,1,1,0,0
1,1,0,0,1,1.0,1,2012,4,1,7,...,0,0,0,0,0,0,1,1,0,0
2,2,0,0,2,1.0,1,2012,4,1,7,...,0,0,0,0,0,0,1,1,0,0
3,3,0,0,3,1.0,1,2012,4,1,7,...,0,0,0,0,0,0,1,1,0,0
4,4,0,1,2,1.0,1,2012,4,1,7,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81081,81081,17615,0,0,1.0,3,2015,3,31,2,...,0,0,0,0,0,0,1,0,0,1
81082,81082,17615,0,2,1.0,3,2015,3,31,2,...,0,0,0,0,0,0,1,0,0,1
81083,81083,17615,0,3,1.0,3,2015,3,31,2,...,0,0,0,0,0,0,1,1,0,0
81084,81084,17615,1,0,1.0,3,2015,3,31,2,...,0,0,0,0,0,0,1,0,0,1


## MODEL 1

We put different betas for the different variables of cost and for the different variables on time

In [12]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_dur_inter=Beta('B_dur_inter',0,None,None,0)
B_n_inter=Beta('B_n_inter',0,None,None,0)

V0 = ASC_cycling + B_time_1 * dur_cycling
V1 = ASC_driving + B_time_2 * dur_driving + B_cost_2 * cost_driving_total
V2 = B_time_3 * (dur_pt_rail +dur_pt_bus +dur_pt_access) + B_dur_inter*(dur_pt_int_total) + B_n_inter*(pt_n_interchanges) + B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * dur_walking

V_1 = {0:V0, 1:V1, 2:V2, 3:V3}
av_1 = {0:1, 1:1, 2:1, 3:1}

In [13]:
logprob = models.loglogit(V_1,av_1,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_1'
results_1 = biogeme.estimate()

In [14]:
# Get the results in a pandas table
pandasResults_1 = results_1.getEstimatedParameters()
print(pandasResults_1)
print('-----------------------------------')
final_LL_1 = results_1.data.logLike
print(f'Final Loglikelihood: {final_LL_1}')

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling -2.286851      0.040535   -56.416980      0.000000
ASC_driving  0.472526      0.020004    23.621788      0.000000
ASC_walking  2.394155      0.035133    68.146407      0.000000
B_cost_2    -0.145912      0.003907   -37.344621      0.000000
B_cost_3    -0.178363      0.008215   -21.712189      0.000000
B_dur_inter -5.195028      0.408679   -12.711755      0.000000
B_n_inter    0.153810      0.051003     3.015704      0.002564
B_time_1    -5.471935      0.120224   -45.514485      0.000000
B_time_2    -6.316319      0.106814   -59.133686      0.000000
B_time_3    -3.168748      0.067329   -47.063725      0.000000
B_time_4    -8.276034      0.102789   -80.514628      0.000000
-----------------------------------
Final Loglikelihood: -69818.87614475124


We can highlight that a negative value in the B_dur_inter makes sense because an increase in the duration of the interchange will mean a decrease in the utility. However, we can't use this medel because tha possitive value of the B_n_inter (number of interchanges) makes no sense (because an increase in the duration of the interchange cannot mean an increase in the utility). We also tested in the development of this assigment the equivalent models with same betas for all times and same for all costs also provoque a possitive result of number of interchanges.

## Model 2

Now we split the variable "dur_pt_int_total" in the sum of "dur_pt_int_waiting" and "dur_pt_int_walking".

In [15]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_dur_walking=Beta('B_dur_walking',0,None,None,0)
B_dur_waiting=Beta('B_dur_waiting',0,None,None,0)
B_n_inter=Beta('B_n_inter',0,None,None,0)

V0 = ASC_cycling + B_time_1 * dur_cycling
V1 = ASC_driving + B_time_2 * dur_driving + B_cost_2 * cost_driving_total
V2 = B_time_3 * (dur_pt_rail +dur_pt_bus +dur_pt_access) + B_dur_waiting*(dur_pt_int_waiting) + B_dur_walking*(dur_pt_int_walking) + B_n_inter*(pt_n_interchanges) + B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * dur_walking

V_2 = {0:V0, 1:V1, 2:V2, 3:V3}
av_2 = {0:1, 1:1, 2:1, 3:1}

In [16]:
logprob = models.loglogit(V_2,av_2,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_2'
results_2 = biogeme.estimate()

In [17]:
# Get the results in a pandas table
pandasResults = results_2.getEstimatedParameters()
print(pandasResults)
print('-----------------------------------')
final_LL_2 = results_2.data.logLike
print(f'Final Loglikelihood: {final_LL_2}')

                  Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling   -2.287255      0.040536   -56.424651  0.000000e+00
ASC_driving    0.471912      0.020007    23.587886  0.000000e+00
ASC_walking    2.393879      0.035134    68.136599  0.000000e+00
B_cost_2      -0.145928      0.003907   -37.354937  0.000000e+00
B_cost_3      -0.179788      0.008259   -21.768169  0.000000e+00
B_dur_waiting -5.538907      0.461713   -11.996434  0.000000e+00
B_dur_walking -4.009316      0.803298    -4.991071  6.004548e-07
B_n_inter      0.168991      0.052184     3.238380  1.202106e-03
B_time_1      -5.455941      0.120515   -45.272051  0.000000e+00
B_time_2      -6.296644      0.107510   -58.567746  0.000000e+00
B_time_3      -3.155566      0.067855   -46.504799  0.000000e+00
B_time_4      -8.270890      0.102880   -80.393470  0.000000e+00
-----------------------------------
Final Loglikelihood: -69817.36450026325


Again, we can't use this medel because tha possitive value of the B_n_inter

## Model 3

In [18]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_dur_inter_norm=Beta('B_dur_inter_norm',0,None,None,0)

V0 = ASC_cycling + B_time_1 * dur_cycling
V1 = ASC_driving + B_time_2 * dur_driving + B_cost_2 * cost_driving_total
V2 = B_time_3 * (dur_pt_rail +dur_pt_bus +dur_pt_access) + B_dur_inter_norm*(dur_pt_int_total/pt_n_interchanges)+ B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * dur_walking

V_3 = {0:V0, 1:V1, 2:V2, 3:V3}
av_3 = {0:1, 1:1, 2:1, 3:1}

In [19]:
logprob = models.loglogit(V_3,av_3,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_3'
results_3 = biogeme.estimate()

In [20]:
# Get the results in a pandas table
pandasResults3 = results_3.getEstimatedParameters()
print(pandasResults3)
print('-----------------------------------')
final_LL_3 = results_3.data.logLike
print(f'Final Loglikelihood: {final_LL_3}')

                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling      -2.341099      0.040469   -57.848793           0.0
ASC_driving       0.404807      0.019390    20.876781           0.0
ASC_walking       2.347125      0.034835    67.377517           0.0
B_cost_2         -0.151064      0.003880   -38.936721           0.0
B_cost_3         -0.200788      0.008026   -25.017280           0.0
B_dur_inter_norm -3.416717      0.193254   -17.679893           0.0
B_time_1         -5.332373      0.119655   -44.564563           0.0
B_time_2         -6.039518      0.104422   -57.837515           0.0
B_time_3         -3.196185      0.066730   -47.897371           0.0
B_time_4         -8.242294      0.102627   -80.313211           0.0
-----------------------------------
Final Loglikelihood: -70008.07153600254


This could be the first possible model due to the negative values of all the betas that represent a negative effect on the utility.

## Model 4

Now we split the variable "dur_pt_int_total/pt_n_interchanges" in the sum of "dur_pt_int_waiting/pt_n_interchanges" and "dur_pt_int_walking/pt_n_interchanges".

In [21]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_dur_walking=Beta('B_dur_walking',0,None,None,0)
B_dur_waiting=Beta('B_dur_waiting',0,None,None,0)

V0 = ASC_cycling + B_time_1 * dur_cycling
V1 = ASC_driving + B_time_2 * dur_driving + B_cost_2 * cost_driving_total
V2 = B_time_3 * (dur_pt_rail +dur_pt_bus +dur_pt_access) + B_dur_waiting*(dur_pt_int_waiting/pt_n_interchanges) + B_dur_walking*(dur_pt_int_walking/pt_n_interchanges) + B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * dur_walking

V_4 = {0:V0, 1:V1, 2:V2, 3:V3}
av_4 = {0:1, 1:1, 2:1, 3:1}

In [22]:
logprob = models.loglogit(V_4,av_4,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_4'
results_4 = biogeme.estimate()

In [23]:
# Get the results in a pandas table
pandasResults4 = results_4.getEstimatedParameters()
print(pandasResults4)
print('-----------------------------------')
final_LL_4 = results_4.data.logLike
print(f'Final Loglikelihood: {final_LL_4}')

                  Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling   -2.341351      0.040477   -57.843431      0.000000
ASC_driving    0.404576      0.019393    20.862342      0.000000
ASC_walking    2.347215      0.034835    67.380329      0.000000
B_cost_2      -0.151052      0.003879   -38.943668      0.000000
B_cost_3      -0.201912      0.008054   -25.068566      0.000000
B_dur_waiting -3.649701      0.242998   -15.019499      0.000000
B_dur_walking -2.080160      0.830309    -2.505284      0.012235
B_time_1      -5.320435      0.119826   -44.401354      0.000000
B_time_2      -6.025352      0.104909   -57.434254      0.000000
B_time_3      -3.184916      0.067233   -47.371277      0.000000
B_time_4      -8.238824      0.102706   -80.217776      0.000000
-----------------------------------
Final Loglikelihood: -70006.81730421167


In [24]:
# Ratio test between Model 4 and Model 3

chi = -2*(final_LL_3 - final_LL_4)

a = tools.likelihood_ratio_test((final_LL_3,10), (final_LL_4,11), significance_level=0.05)

print(a)
print(chi)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=2.50846358173294, threshold=3.841458820694124)
2.50846358173294


We do the Ratio test between Model 4 and Model 3 to see if there is a significant improvement with a 5% of level of significance

As we can see, the Model 4 is not significantly better than the Model 3.

## MODEL 5

dur interchamges + separation between different cases of number of interchanges

In [25]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_dur_inter=Beta('B_dur_inter',0,None,None,0)
B_1_inter=Beta('B_1_inter',0,None,None,0)
B_n_inter = Beta('B_n_inter',0,None,None,0)

V0 = ASC_cycling + B_time_1 * dur_cycling
V1 = ASC_driving + B_time_2 * dur_driving + B_cost_2 * cost_driving_total
V2 = B_time_3 * (dur_pt_rail +dur_pt_bus +dur_pt_access) +B_dur_inter*dur_pt_int_total +B_n_inter*(more_than_one_inter) +B_1_inter*(one_inter)+ B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * dur_walking

V_5 = {0:V0, 1:V1, 2:V2, 3:V3}
av_5 = {0:1, 1:1, 2:1, 3:1}

In [26]:
logprob = models.loglogit(V_5,av_5,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_5'
results_5 = biogeme.estimate()

In [27]:
# Get the results in a pandas table
pandasResults_5 = results_5.getEstimatedParameters()
print(pandasResults_5)
print('-----------------------------------')
final_LL_5 = results_5.data.logLike
print(f'Final Loglikelihood: {final_LL_5}')

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling -2.278938      0.040246   -56.624957  0.000000e+00
ASC_driving  0.485432      0.020082    24.172937  0.000000e+00
ASC_walking  2.397219      0.035239    68.027751  0.000000e+00
B_1_inter    0.339504      0.050383     6.738461  1.600742e-11
B_cost_2    -0.144740      0.003915   -36.966485  0.000000e+00
B_cost_3    -0.180690      0.008187   -22.071711  0.000000e+00
B_dur_inter -5.553809      0.389534   -14.257588  0.000000e+00
B_n_inter    0.224354      0.104769     2.141421  3.224008e-02
B_time_1    -5.487271      0.119383   -45.963664  0.000000e+00
B_time_2    -6.368854      0.107135   -59.446902  0.000000e+00
B_time_3    -3.218428      0.067608   -47.604576  0.000000e+00
B_time_4    -8.279532      0.103014   -80.372954  0.000000e+00
-----------------------------------
Final Loglikelihood: -69770.153020523


Same problem, beta values are positive

## Model 6

Now we split the variable "dur_pt_int_total" in the sum of "dur_pt_int_waiting" and "dur_pt_int_walking".

In [28]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_dur_walking=Beta('B_dur_walking',0,None,None,0)
B_dur_waiting=Beta('B_dur_waiting',0,None,None,0)
B_1_inter=Beta('B_1_inter',0,None,None,0)
B_n_inter = Beta('B_n_inter',0,None,None,0)

V0 = ASC_cycling + B_time_1 * dur_cycling
V1 = ASC_driving + B_time_2 * dur_driving + B_cost_2 * cost_driving_total
V2 = B_time_3 * (dur_pt_rail +dur_pt_bus +dur_pt_access) + B_dur_waiting*(dur_pt_int_waiting) + B_dur_walking*(dur_pt_int_walking) + B_n_inter*(more_than_one_inter) +B_1_inter*(one_inter) + B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * dur_walking

V_6 = {0:V0, 1:V1, 2:V2, 3:V3}
av_6 = {0:1, 1:1, 2:1, 3:1}

In [29]:
logprob = models.loglogit(V_6,av_6,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_6'
results_6 = biogeme.estimate()

In [30]:
# Get the results in a pandas table
pandasResults = results_6.getEstimatedParameters()
print(pandasResults)
print('-----------------------------------')
final_LL_6 = results_6.data.logLike
print(f'Final Loglikelihood: {final_LL_6}')

                  Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling   -2.279056      0.040246   -56.627987  0.000000e+00
ASC_driving    0.484993      0.020084    24.148240  0.000000e+00
ASC_walking    2.397102      0.035239    68.023514  0.000000e+00
B_1_inter      0.354098      0.051369     6.893214  5.454526e-12
B_cost_2      -0.144769      0.003915   -36.981590  0.000000e+00
B_cost_3      -0.182418      0.008229   -22.167523  0.000000e+00
B_dur_waiting -5.932932      0.440193   -13.478029  0.000000e+00
B_dur_walking -4.063761      0.795978    -5.105371  3.301455e-07
B_n_inter      0.251273      0.106740     2.354080  1.856860e-02
B_time_1      -5.467506      0.119634   -45.702125  0.000000e+00
B_time_2      -6.344226      0.107777   -58.864241  0.000000e+00
B_time_3      -3.201545      0.068124   -46.995677  0.000000e+00
B_time_4      -8.272959      0.103096   -80.245261  0.000000e+00
-----------------------------------
Final Loglikelihood: -69767.7452154655


Again, we can't use this model because of the possitive values of the betas B_1_inter and B_n_inter.

## Model 7

In addition to the approach in Model 5, the reason why we are getting positive betas could be also due to the fact that the scale of global time and the scale of number of interchanges is not the same so we rewrite duration of public transport with a logarithm transformation. However, we can't do this transformation with the variable of the number of interchanges because too many values of this variable are equal to zero and will lead us to an error of log(0) in the code.

Log-transformed variables are commonly used in multinomial logit models to capture non-linear relationships between variables and the likelihood of selecting a certain alternative resulting in better predictions.

In our example of log-transformed duration time, the utility of a mode of transportation decreases as trip time increases, but the correlation between travel time and utility is often non-linear. The same applies for costs.

The coefficients associated with the log-transformed variable can be interpreted as the change in log-odds of selecting a certain mode of transportation for a one percent change in travel time.


In [12]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_1_inter=Beta('B_1_inter',0,None,None,0)
B_n_inter = Beta('B_n_inter',0,None,None,0)
B_dur_inter = Beta('B_dur_inter',0,None,None,0)

V0 = ASC_cycling + B_time_1 * log(dur_cycling)
V1 = ASC_driving + B_time_2 * log(dur_driving) + B_cost_2 * cost_driving_total
V2 = B_time_3 * log(dur_pt_rail +dur_pt_bus +dur_pt_access)+ B_dur_inter*log((1+dur_pt_int_total)/(1+pt_n_interchanges)) +B_n_inter*(more_than_one_inter) +B_1_inter*(one_inter)+ B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * log(dur_walking)

V_7 = {0:V0, 1:V1, 2:V2, 3:V3}
av_7 = {0:1, 1:1, 2:1, 3:1}

In [13]:
logprob = models.loglogit(V_7,av_7,choice)
biogeme = bio.BIOGEME (database, logprob)
biogeme.modelName='Model_7'
results_7 = biogeme.estimate()

[09:29:47] < Warning >   Cannot read file __Model_7.iter. Statement is ignored.


In [14]:
# Get the results in a pandas table
pandasResults7 = results_7.getEstimatedParameters()
print(pandasResults7)
print('-----------------------------------')
final_LL_7 = results_7.data.logLike
print(f'Final Loglikelihood: {final_LL_7}')

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling -3.967382      0.048162   -82.376296  0.000000e+00
ASC_driving -1.295666      0.034053   -38.048194  0.000000e+00
ASC_walking -1.823593      0.036362   -50.151150  0.000000e+00
B_1_inter   -1.729400      0.181208    -9.543724  0.000000e+00
B_cost_2    -0.151117      0.003889   -38.861177  0.000000e+00
B_cost_3    -0.132424      0.006935   -19.095843  0.000000e+00
B_dur_inter -2.211748      0.309656    -7.142590  9.159340e-13
B_n_inter   -2.870055      0.282932   -10.143962  0.000000e+00
B_time_1    -2.200435      0.031338   -70.216984  0.000000e+00
B_time_2    -2.103768      0.025712   -81.819056  0.000000e+00
B_time_3    -1.793319      0.028960   -61.924065  0.000000e+00
B_time_4    -4.359137      0.031964  -136.376859  0.000000e+00
-----------------------------------
Final Loglikelihood: -68605.81018867838


This could be the first possible model due to the negative values of all the betas that represent a negative on the utility. And also a better log likelihood


As we have seen, there ara only three possible models to select (because they have all negative betas): Model 3, Model 4 and Model 7. With the Ratio test done before, we set that the Model 3 was better than the Model 4, as the Model 4 was not significantly better than Model 3. Then, we are going to do no the Ratio test between Model 3 and 7.

In [3]:
# Ratio test
final_LL_3=-70008.0714174418
final_LL_7=-68605.81346897618
chi = -2*(final_LL_3 - final_LL_7)

a = tools.likelihood_ratio_test((final_LL_3,10), (final_LL_7,11), significance_level=0.05)

print(a)
print(chi)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=2804.5158969312324, threshold=3.841458820694124)
2804.5158969312324


So the Model is the best MNL because it is significanly better than the rest of the possible models.

In [29]:
#Calculation of elasticities of the Model 7

prob_0 = models.logit(V_7,av_7,0)
prob_1 = models.logit(V_7,av_7,1)
prob_2 = models.logit(V_7,av_7,2)
prob_3 = models.logit(V_7,av_7,3)

#Direct and Cross Elasticities of each alternative (only the ones for our study)
more_than_one_inter
one_inter

elas_2_one_inter = Derive(prob_2,'one_inter') * one_inter / prob_2 
elas_2_n_inter = Derive(prob_2,'more_than_one_inter') * more_than_one_inter / prob_2 

elastcross_0_one_inter = Derive(prob_0,'one_inter') * one_inter / prob_0 
elastcross_1_one_inter = Derive(prob_1,'one_inter') * one_inter / prob_1 
elastcross_3_one_inter = Derive(prob_3,'one_inter') * one_inter / prob_3

elastcross_0_n_inter = Derive(prob_0,'more_than_one_inter') * more_than_one_inter / prob_0
elastcross_1_n_inter = Derive(prob_1,'more_than_one_inter') * more_than_one_inter / prob_1
elastcross_3_n_inter = Derive(prob_3,'more_than_one_inter') * more_than_one_inter / prob_3



elas_2_dur_inter = Derive(prob_2,'dur_pt_int_total') * dur_pt_int_total / prob_2 

elastcross_0_dur_inter = Derive(prob_0,'dur_pt_int_total') * dur_pt_int_total / prob_0 
elastcross_1_dur_inter = Derive(prob_1,'dur_pt_int_total') * dur_pt_int_total / prob_1 
elastcross_3_dur_inter = Derive(prob_3,'dur_pt_int_total') * dur_pt_int_total / prob_3



simulate = {
    'prob.0': prob_0,
    'prob.1': prob_1,
    'prob.2': prob_2,
    'prob.3': prob_3,
    'elas_2_one_inter': elas_2_one_inter,
    'elas_2_n_inter': elas_2_n_inter,
    'elastcross_0_one_inter': elastcross_0_one_inter,
    'elastcross_1_one_inter': elastcross_1_one_inter,
    'elastcross_3_one_inter': elastcross_3_one_inter,
    'elastcross_0_n_inter': elastcross_0_n_inter,
    'elastcross_1_n_inter': elastcross_1_n_inter,
    'elastcross_3_n_inter': elastcross_3_n_inter,
    'elas_2_dur_inter': elas_2_dur_inter,
    'elastcross_0_dur_inter': elastcross_0_dur_inter,
    'elastcross_1_dur_inter': elastcross_1_dur_inter,
    'elastcross_3_dur_inter': elastcross_3_dur_inter,
}

biosim = bio.BIOGEME(database, simulate)
biosim.modelName = 'elasticities_Model_7'


# Perform the simulation
simresults = biosim.simulate(results_7.getBetaValues())
simresults2=simresults

denominator_0 = simresults['prob.0'].sum()
denominator_1 = simresults['prob.1'].sum()
denominator_2 = simresults['prob.2'].sum()
denominator_3 = simresults['prob.3'].sum()

elas_2_one_inter = (simresults['prob.2'] * simresults['elas_2_one_inter'] / denominator_2).sum()
elas_2_n_inter = (simresults['prob.2'] * simresults['elas_2_n_inter'] / denominator_2).sum()

elastcross_0_one_inter = (simresults['prob.0'] * simresults['elastcross_0_one_inter'] / denominator_0).sum()
elastcross_1_one_inter  = (simresults['prob.1'] * simresults['elastcross_1_one_inter'] / denominator_1).sum()
elastcross_3_one_inter = (simresults['prob.3'] * simresults['elastcross_3_one_inter'] / denominator_3).sum()

elastcross_0_n_inter = (simresults['prob.0'] * simresults['elastcross_0_n_inter'] / denominator_0).sum()
elastcross_1_n_inter = (simresults['prob.1'] * simresults['elastcross_1_n_inter'] / denominator_1).sum()
elastcross_3_n_inter = (simresults['prob.3'] * simresults['elastcross_3_n_inter'] / denominator_3).sum()




elas_2_dur_inter = (simresults['prob.2'] * simresults['elas_2_dur_inter'] / denominator_2).sum()

elastcross_0_dur_inter = (simresults['prob.0'] * simresults['elastcross_0_dur_inter'] / denominator_0).sum()
elastcross_1_dur_inter  = (simresults['prob.1'] * simresults['elastcross_1_dur_inter'] / denominator_1).sum()
elastcross_3_dur_inter = (simresults['prob.3'] * simresults['elastcross_3_dur_inter'] / denominator_3).sum()



simresults_1=simresults



In [30]:
print(
f'The average elasticity of the share of alternative 2 with respect to one interchange: {elas_2_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 2 with respect to more than one interchange: {elas_2_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 0 with respect to one interchange: {elastcross_0_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 1 with respect to one interchange: {elastcross_1_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 3 with respect to one interchange: {elastcross_3_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 0 with respect to more than one interchange: {elastcross_0_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 1 with respect to more than one interchange: {elastcross_1_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 3 with respect to more than one interchange: {elastcross_3_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 2 with respect to duration of interchange: {elas_2_dur_inter:.3f} \n'
f'The average elasticity of the share of alternative 0 with respect to duration of interchange: {elastcross_0_dur_inter:.3f} \n'
f'The average elasticity of the share of alternative 1 with respect to duration of interchange: {elastcross_1_dur_inter:.3f} \n'
f'The average elasticity of the share of alternative 3 with respect to duration of interchange: {elastcross_3_dur_inter:.3f} \n')

The average elasticity of the share of alternative 2 with respect to one interchange: -0.226 
The average elasticity of the share of alternative 2 with respect to more than one interchange: -0.099 
The average elasticity of the share of alternative 0 with respect to one interchange: 0.184 
The average elasticity of the share of alternative 1 with respect to one interchange: 0.163 
The average elasticity of the share of alternative 3 with respect to one interchange: 0.014 
The average elasticity of the share of alternative 0 with respect to more than one interchange: 0.079 
The average elasticity of the share of alternative 1 with respect to more than one interchange: 0.073 
The average elasticity of the share of alternative 3 with respect to more than one interchange: 0.001 
The average elasticity of the share of alternative 2 with respect to duration of interchange: -0.046 
The average elasticity of the share of alternative 0 with respect to duration of interchange: 0.036 
The average

In [22]:
# Calculation of Value of Time

TOTAL_DUR_PT=(df['dur_pt_int_total'])+(df['dur_pt_rail'])+(df['dur_pt_bus'])+(df['dur_pt_access'])


VOT_PT_INTERCHANGES = np.mean((pandasResults7.loc['B_time_3']['Value'])*(df['cost_transit']+0.00001)/(pandasResults7.loc['B_cost_3']['Value']*TOTAL_DUR_PT))
VOT_CAR = np.mean(pandasResults7.loc['B_time_2']['Value']*(df['cost_driving_total']+0.00001)/((pandasResults7.loc['B_cost_2']['Value'])*df['dur_driving']))

print(f"The value of time in the interchanges of Public Transport is: {VOT_PT_INTERCHANGES:.2f}")
print(f"The value of time for Driving is: {VOT_CAR:.2f}")

The value of time in the interchanges of Public Transport is: 55.91
The value of time for Driving is: 88.33


# EXTENDED MODEL

Development of a Nested Logit Model

### First logical nested model: Motor(pt and car), No Motor (walk and cycle)

In [19]:
ASC_cycling = Beta('ASC_cycling',0,None,None,0)
ASC_driving = Beta('ASC_driving',0,None,None,0)
ASC_walking = Beta('ASC_walking',0,None,None,0)

B_time_1 = Beta('B_time_1',0,None,None,0)
B_time_2 = Beta('B_time_2',0,None,None,0)
B_time_3 = Beta('B_time_3',0,None,None,0)
B_time_4 = Beta('B_time_4',0,None,None,0)
B_cost_2 = Beta('B_cost_2',0,None,None,0)
B_cost_3 = Beta('B_cost_3',0,None,None,0)
B_1_inter=Beta('B_1_inter',0,None,None,0)
B_n_inter = Beta('B_n_inter',0,None,None,0)
B_dur_inter = Beta('B_dur_inter',0,None,None,0)

V0 = ASC_cycling + B_time_1 * log(dur_cycling)
V1 = ASC_driving + B_time_2 * log(dur_driving) + B_cost_2 * cost_driving_total
V2 = B_time_3 * log(dur_pt_rail +dur_pt_bus +dur_pt_access) +B_dur_inter*log((1+dur_pt_int_total)/(1+pt_n_interchanges))+B_n_inter*(more_than_one_inter) +B_1_inter*(one_inter)+ B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * log(dur_walking)

V_NL1 = {0:V0, 1:V1, 2:V2, 3:V3}
av_NL1 = {0:1, 1:1, 2:1, 3:1}

#1: Nest parameters
NEST_MOTOR=Beta('NEST_MOTOR',1,1,10,0)
NEST_NOMOTOR=Beta('NEST_NOMOTOR',1,1,10,0)
#2: List of alternatives
NO_MOTOR=NEST_NOMOTOR, [1,2]
MOTOR=NEST_MOTOR, [0,3]

nests=NO_MOTOR, MOTOR

logprob = models.lognested(V_NL1, av_NL1, nests, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'NL_1'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av_NL1)

# Estimate the parameters
results_NL1 = biogeme.estimate()

# Get the results in a pandas table
pandasResults_NL1 = results_NL1.getEstimatedParameters()
pandasResults2=pandasResults_NL1
print(pandasResults_NL1)
print(results_NL1)

[15:18:12] < Warning >   Cannot read file __NL_1.iter. Statement is ignored.
                 Value  Active bound  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling  -3.958183           0.0      0.044277   -89.396056  0.000000e+00
ASC_driving  -0.894351           0.0      0.036243   -24.676220  0.000000e+00
ASC_walking  -2.484792           0.0      0.053225   -46.684744  0.000000e+00
B_1_inter    -1.086259           0.0      0.116674    -9.310207  0.000000e+00
B_cost_2     -0.083492           0.0      0.004387   -19.033781  0.000000e+00
B_cost_3     -0.081718           0.0      0.005664   -14.428789  0.000000e+00
B_dur_inter  -1.352359           0.0      0.194891    -6.939068  3.947065e-12
B_n_inter    -1.814345           0.0      0.183100    -9.909022  0.000000e+00
B_time_1     -1.600702           0.0      0.051611   -31.014943  0.000000e+00
B_time_2     -1.401066           0.0      0.051071   -27.433517  0.000000e+00
B_time_3     -1.197787           0.0      0.045673   -26.225322  

Having into account that Biogeme gives us the inverse of the mu's values:

In [39]:
MU_NOMOTOR = 1/1.711187
MU_NOMOTOR

0.5843896663544078

### Second logical nested model: Possible interchanges (pt), No interchanges (walk, cycle and car)

In [40]:
#1: Nest parameters
NEST_INTERCHANGES=Beta('NEST_INTERCHANGES',1,1,10,0)
NEST_NO_INTERCHANGES=Beta('NEST_NO_INTERCHANGES',1,1,10,0)
#2: List of alternatives
INTERCHANGES=NEST_INTERCHANGES, [2]
NO_INTERCHANGES=NEST_NO_INTERCHANGES, [0,1,3]

nests= INTERCHANGES, NO_INTERCHANGES

logprob = models.lognested(V_NL1, av_NL1, nests, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'NL_2'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av_NL1)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
pandasResults2=pandasResults
print(pandasResults)
print(results)

                         Value  Active bound  Rob. Std err   Rob. t-test  \
ASC_cycling          -3.969769           0.0  7.706490e-02 -5.151202e+01   
ASC_driving          -1.294232           0.0  3.486087e-02 -3.712564e+01   
ASC_walking          -1.824226           0.0  6.104648e-02 -2.988258e+01   
B_1_inter            -0.445393           0.0  2.398194e-02 -1.857203e+01   
B_cost_2             -0.151754           0.0  3.917787e-03 -3.873455e+01   
B_cost_3             -0.132738           0.0  7.015095e-03 -1.892184e+01   
B_n_inter            -0.860486           0.0  4.183445e-02 -2.056884e+01   
B_time_1             -2.202113           0.0  3.409953e-02 -6.457900e+01   
B_time_2             -2.104048           0.0  2.872055e-02 -7.325931e+01   
B_time_3             -1.794682           0.0  3.475136e-02 -5.164350e+01   
B_time_4             -4.360960           0.0  6.261349e-02 -6.964889e+01   
NEST_INTERCHANGES     1.000000           1.0  1.075010e-18  9.302240e+17   
NEST_NO_INTE

### Third logical nested model: Cars (car), Others (walk, cycle and pt)

In [41]:
#1: Nest parameters
NEST_CARS=Beta('NEST_CARS',1,1,10,0)
NEST_OTHERS=Beta('NEST_OTHERS',1,1,10,0)
#2: List of alternatives
CARS=NEST_CARS, [1]
OTHERS=NEST_OTHERS, [0,2,3]

nests= CARS, OTHERS

logprob = models.lognested(V_NL1, av_NL1, nests, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'NL_3'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av_NL1)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
pandasResults2=pandasResults
print(pandasResults)
print(results)

                Value  Active bound  Rob. Std err   Rob. t-test  Rob. p-value
ASC_cycling -3.969170           0.0  8.203512e-02 -4.838379e+01           0.0
ASC_driving -1.294248           0.0  3.683796e-02 -3.513355e+01           0.0
ASC_walking -1.824013           0.0  5.537230e-02 -3.294090e+01           0.0
B_1_inter   -0.445360           0.0  2.413723e-02 -1.845116e+01           0.0
B_cost_2    -0.151739           0.0  3.979132e-03 -3.813379e+01           0.0
B_cost_3    -0.132733           0.0  7.019459e-03 -1.890934e+01           0.0
B_n_inter   -0.860360           0.0  4.392223e-02 -1.958826e+01           0.0
B_time_1    -2.201990           0.0  3.710960e-02 -5.933746e+01           0.0
B_time_2    -2.104215           0.0  3.203315e-02 -6.568868e+01           0.0
B_time_3    -1.794921           0.0  2.992414e-02 -5.998236e+01           0.0
B_time_4    -4.361107           0.0  6.164098e-02 -7.075012e+01           0.0
NEST_CARS    1.000000           1.0  7.871237e-19  1.270448e+18 

### Forth logical nested model: Ecological (cycle, walk), pt, car.

In [42]:
#1: Nest parameters
NEST_ECOLOGICAL=Beta('NEST_ECOLOGICAL',1,1,10,0)
CAR=Beta('CAR',1,1,10,0)
PT=Beta('PT',1,1,10,0)
#2: List of alternatives
ECOLOGICAL=NEST_ECOLOGICAL, [0, 3]
CAR=1, [1]
PT=2, [2]

nests= ECOLOGICAL,CAR,PT

logprob = models.lognested(V_NL1, av_NL1, nests, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'NL_4'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av_NL1)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
pandasResults2=pandasResults
print(pandasResults)
print(results)

                    Value  Active bound  Rob. Std err  Rob. t-test  \
ASC_cycling     -3.969611           0.0      0.047543   -83.495198   
ASC_driving     -1.294287           0.0      0.034256   -37.782462   
ASC_walking     -1.824132           0.0      0.042043   -43.387675   
B_1_inter       -0.445441           0.0      0.023758   -18.748821   
B_cost_2        -0.151748           0.0      0.003898   -38.934067   
B_cost_3        -0.132723           0.0      0.006924   -19.169681   
B_n_inter       -0.860627           0.0      0.041713   -20.631915   
B_time_1        -2.202180           0.0      0.041236   -53.404379   
B_time_2        -2.104204           0.0      0.026348   -79.860602   
B_time_3        -1.794871           0.0      0.029561   -60.718247   
B_time_4        -4.361155           0.0      0.038822  -112.338546   
NEST_ECOLOGICAL  1.000000           1.0      0.039178    25.524261   

                 Rob. p-value  
ASC_cycling               0.0  
ASC_driving              

So, the first logical nested model is the one showing and improvement in comparison with the Model 7 (in terms of the LogLikelihood) and also having a mu between 0 and 1 for the nest of No Motor the mu equal to 1 in the nest of Motor, that means that maximum substitution takes places inside the nest of No Motor whereas Public Transport and Car don't constitute a nest because it doesn't exist substitution between them.

In [2]:
# Ratio test between NL1 and MNL7.
final_LL_7=-68605.81346897618
final_LL_NL1=-68471.01
chi = -2*(final_LL_7 - final_LL_NL1)

a = tools.likelihood_ratio_test((final_LL_7,11), (final_LL_NL1,13), significance_level=0.05)

print(a)
print(chi)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=269.60693795236875, threshold=5.991464547107979)
269.60693795236875


# ANALYSIS OF THE RESULTS OF NL1

In [27]:
#Calculation of elasticities of the Model NL1

prob_0 = models.logit(V_NL1,av_NL1,0)
prob_1 = models.logit(V_NL1,av_NL1,1)
prob_2 = models.logit(V_NL1,av_NL1,2)
prob_3 = models.logit(V_NL1,av_NL1,3)

#Direct and Cross Elasticities of each alternative (only the ones for our study)
elas_2_one_inter = Derive(prob_2,'one_inter') * one_inter / prob_2 
elas_2_n_inter = Derive(prob_2,'more_than_one_inter') * more_than_one_inter / prob_2 

elastcross_0_one_inter = Derive(prob_0,'one_inter') * one_inter / prob_0 
elastcross_1_one_inter = Derive(prob_1,'one_inter') * one_inter / prob_1 
elastcross_3_one_inter = Derive(prob_3,'one_inter') * one_inter / prob_3

elastcross_0_n_inter = Derive(prob_0,'more_than_one_inter') * more_than_one_inter / prob_0
elastcross_1_n_inter = Derive(prob_1,'more_than_one_inter') * more_than_one_inter / prob_1
elastcross_3_n_inter = Derive(prob_3,'more_than_one_inter') * more_than_one_inter / prob_3



elas_2_dur_inter = Derive(prob_2,'dur_pt_int_total') * dur_pt_int_total / prob_2 

elastcross_0_dur_inter = Derive(prob_0,'dur_pt_int_total') * dur_pt_int_total / prob_0 
elastcross_1_dur_inter = Derive(prob_1,'dur_pt_int_total') * dur_pt_int_total / prob_1 
elastcross_3_dur_inter = Derive(prob_3,'dur_pt_int_total') * dur_pt_int_total / prob_3



simulate = {
    'prob.0': prob_0,
    'prob.1': prob_1,
    'prob.2': prob_2,
    'prob.3': prob_3,
    'elas_2_one_inter': elas_2_one_inter,
    'elas_2_n_inter': elas_2_n_inter,
    'elastcross_0_one_inter': elastcross_0_one_inter,
    'elastcross_1_one_inter': elastcross_1_one_inter,
    'elastcross_3_one_inter': elastcross_3_one_inter,
    'elastcross_0_n_inter': elastcross_0_n_inter,
    'elastcross_1_n_inter': elastcross_1_n_inter,
    'elastcross_3_n_inter': elastcross_3_n_inter,
    'elas_2_dur_inter': elas_2_dur_inter,
    'elastcross_0_dur_inter': elastcross_0_dur_inter,
    'elastcross_1_dur_inter': elastcross_1_dur_inter,
    'elastcross_3_dur_inter': elastcross_3_dur_inter,
}

biosim = bio.BIOGEME(database, simulate)
biosim.modelName = 'elasticities_Model_NL1'


# Perform the simulation
simresults = biosim.simulate(results_NL1.getBetaValues())
simresults2=simresults

denominator_0 = simresults['prob.0'].sum()
denominator_1 = simresults['prob.1'].sum()
denominator_2 = simresults['prob.2'].sum()
denominator_3 = simresults['prob.3'].sum()

elas_2_one_inter = (simresults['prob.2'] * simresults['elas_2_one_inter'] / denominator_2).sum()
elas_2_n_inter = (simresults['prob.2'] * simresults['elas_2_n_inter'] / denominator_2).sum()

elastcross_0_one_inter = (simresults['prob.0'] * simresults['elastcross_0_one_inter'] / denominator_0).sum()
elastcross_1_one_inter  = (simresults['prob.1'] * simresults['elastcross_1_one_inter'] / denominator_1).sum()
elastcross_3_one_inter = (simresults['prob.3'] * simresults['elastcross_3_one_inter'] / denominator_3).sum()

elastcross_0_n_inter = (simresults['prob.0'] * simresults['elastcross_0_n_inter'] / denominator_0).sum()
elastcross_1_n_inter = (simresults['prob.1'] * simresults['elastcross_1_n_inter'] / denominator_1).sum()
elastcross_3_n_inter = (simresults['prob.3'] * simresults['elastcross_3_n_inter'] / denominator_3).sum()




elas_2_dur_inter = (simresults['prob.2'] * simresults['elas_2_dur_inter'] / denominator_2).sum()

elastcross_0_dur_inter = (simresults['prob.0'] * simresults['elastcross_0_dur_inter'] / denominator_0).sum()
elastcross_1_dur_inter  = (simresults['prob.1'] * simresults['elastcross_1_dur_inter'] / denominator_1).sum()
elastcross_3_dur_inter = (simresults['prob.3'] * simresults['elastcross_3_dur_inter'] / denominator_3).sum()


simresults_1=simresults


[16:10:44] < Warning >   Parameter NEST_MOTOR not present in the model
[16:10:44] < Warning >   Parameter NEST_NOMOTOR not present in the model


In [28]:
print(
f'The average elasticity of the share of alternative 2 with respect to one interchange: {elas_2_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 2 with respect to more than one interchange: {elas_2_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 0 with respect to one interchange: {elastcross_0_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 1 with respect to one interchange: {elastcross_1_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 3 with respect to one interchange: {elastcross_3_one_inter:.3f} \n'
f'The average elasticity of the share of alternative 0 with respect to more than one interchange: {elastcross_0_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 1 with respect to more than one interchange: {elastcross_1_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 3 with respect to more than one interchange: {elastcross_3_n_inter:.3f} \n'
f'The average elasticity of the share of alternative 2 with respect to duration of interchange: {elas_2_dur_inter:.3f} \n'
f'The average elasticity of the share of alternative 0 with respect to duration of interchange: {elastcross_0_dur_inter:.3f} \n'
f'The average elasticity of the share of alternative 1 with respect to duration of interchange: {elastcross_1_dur_inter:.3f} \n'
f'The average elasticity of the share of alternative 3 with respect to duration of interchange: {elastcross_3_dur_inter:.3f} \n')

The average elasticity of the share of alternative 2 with respect to one interchange: -0.149 
The average elasticity of the share of alternative 2 with respect to more than one interchange: -0.065 
The average elasticity of the share of alternative 0 with respect to one interchange: 0.112 
The average elasticity of the share of alternative 1 with respect to one interchange: 0.121 
The average elasticity of the share of alternative 3 with respect to one interchange: 0.009 
The average elasticity of the share of alternative 0 with respect to more than one interchange: 0.043 
The average elasticity of the share of alternative 1 with respect to more than one interchange: 0.054 
The average elasticity of the share of alternative 3 with respect to more than one interchange: 0.001 
The average elasticity of the share of alternative 2 with respect to duration of interchange: -0.029 
The average elasticity of the share of alternative 0 with respect to duration of interchange: 0.021 
The average

In [25]:
# Calculation of Value of Time

TOTAL_DUR_PT=(df['dur_pt_rail'])+(df['dur_pt_bus'])+(df['dur_pt_access'])


VOT_PT_INTERCHANGES = np.mean((pandasResults_NL1.loc['B_time_3']['Value'])/(pandasResults_NL1.loc['B_cost_3']['Value']*TOTAL_DUR_PT))
VOT_CAR = np.mean(pandasResults_NL1.loc['B_time_2']['Value']/((pandasResults_NL1.loc['B_cost_2']['Value'])*df['dur_driving']))

print(f"The value of time in the interchanges of Public Transport is: {VOT_PT_INTERCHANGES:.2f}")
print(f"The value of time for Driving is: {VOT_CAR:.2f}")

The value of time in the interchanges of Public Transport is: 53.85
The value of time for Driving is: 122.12


## MIXED LOGIT

In [53]:
from biogeme.expressions import bioDraws
from biogeme.expressions import MonteCarlo

In [54]:
ASC_cycling = Beta('ASC_cycling', 0, None, None, 0)
ASC_driving = Beta('ASC_driving', 0, None, None, 0)
ASC_walking = Beta('ASC_walking', 0, None, None, 0)
B_time_1 = Beta('B_time_1', 0, None, None, 0)
B_time_2 = Beta('B_time_2', 0, None, None, 0)
B_time_3 = Beta('B_time_3', 0, None, None, 0)
B_time_4 = Beta('B_time_4', 0, None, None, 0)
B_cost_2 = Beta('B_cost_2', 0, None, None, 0)
B_cost_3 = Beta('B_cost_3', 0, None, None, 0)
B_1_inter = Beta('B_1_inter', 0, None, None, 0)
B_n_inter = Beta('B_n_inter', 0, None, None, 0)
B_dur_inter = Beta('B_dur_inter', 0, None, None, 0)

B_time_1_s = Beta('B_time_1_s', -2.202082, None, None, 0)
B_time_2_s = Beta('B_time_2_s', -2.104149, None, None, 0)
B_time_3_s = Beta('B_time_3_s', -1.794938, None, None, 0)
#B_time_4_s = Beta('B_time_4_s', -4.361066, None, None, 0)
#B_cost_2_s = Beta('B_cost_2_s', -0.151756, None, None, 0)
#B_cost_3_s = Beta('B_cost_3_s', -0.132690, None, None, 0)
#B_1_inter_s = Beta('B_1_inter_s', -0.445403, None, None, 0)
#B_n_inter_s = Beta('B_n_inter_s', -0.860717, None, None, 0)

B_time_1_rnd = B_time_1 + B_time_1_s*bioDraws('B_time_1_rnd', "NORMAL")
B_time_2_rnd = B_time_2 + B_time_2_s*bioDraws('B_time_2_rnd', "NORMAL")
B_time_3_rnd = B_time_3 + B_time_3_s*bioDraws('B_time_3_rnd', "NORMAL")
#B_time_4_rnd = B_time_4 + B_time_4_s*bioDraws('B_time_4_rnd', "NORMAL")
#B_cost_2_rnd = B_cost_2 + B_cost_2_s*bioDraws('B_cost_2_rnd', "NORMAL")
#B_cost_3_rnd = B_cost_3 + B_cost_3_s*bioDraws('B_cost_3_rnd', "NORMAL")
#B_1_inter_rnd = B_1_inter + B_1_inter_s*bioDraws('B_1_inter_rnd', "NORMAL")
#B_n_inter_rnd = B_n_inter + B_n_inter_s*bioDraws('B_n_inter_rnd', "NORMAL")

# Define the utility functions for each alternative
V0 = ASC_cycling + B_time_1_rnd * log(dur_cycling)
V1 = ASC_driving + B_time_2_rnd * log(dur_driving) + B_cost_2 * cost_driving_total
V2 = B_time_3_rnd * log(dur_pt_rail + dur_pt_bus + dur_pt_access) + B_dur_inter*log((1+dur_pt_int_total)/(1+pt_n_interchanges))+ B_n_inter * (more_than_one_inter) + B_1_inter * (one_inter) + B_cost_3 * cost_transit
V3 = ASC_walking + B_time_4 * log(dur_walking)


V_ML = {0:V0, 1:V1, 2:V2, 3:V3}
av_ML = {0:1, 1:1, 2:1, 3:1}



In [55]:
# Without panel data (RC)
prob = models.logit(V_ML, av_ML, choice)
logprob = log(MonteCarlo(prob))
biogeme = bio.BIOGEME(database, logprob, numberOfDraws=20)
biogeme.modelName='Model_ML'
results_ML = biogeme.estimate()

In [56]:
pandasResultsML = results_ML.getEstimatedParameters()
print(pandasResultsML)
print('-----------------------------------')
final_LL_ML = results_ML.data.logLike
print(f'Final Loglikelihood: {final_LL_ML}')

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_cycling -3.965712      0.048247   -82.195572  0.000000e+00
ASC_driving -1.297262      0.034104   -38.038096  0.000000e+00
ASC_walking -1.824104      0.036399   -50.113810  0.000000e+00
B_1_inter   -1.720694      0.181202    -9.495996  0.000000e+00
B_cost_2    -0.151163      0.003893   -38.833902  0.000000e+00
B_cost_3    -0.132635      0.006942   -19.105712  0.000000e+00
B_dur_inter -2.196034      0.309641    -7.092202  1.320055e-12
B_n_inter   -2.855967      0.282929   -10.094282  0.000000e+00
B_time_1    -2.196787      0.032475   -67.646291  0.000000e+00
B_time_1_s  -0.072809      0.049067    -1.483866  1.378445e-01
B_time_2    -2.106835      0.025899   -81.348617  0.000000e+00
B_time_2_s   0.051052      0.029934     1.705509  8.809947e-02
B_time_3    -1.796568      0.029211   -61.502740  0.000000e+00
B_time_3_s   0.027551      0.026588     1.036233  3.000934e-01
B_time_4    -4.365108      0.032522  -134.218885  0.000